# Convert dl2 to dl3

First of all we import packages and create the logger

In [1]:
# packages
import numpy as np 
import matplotlib.pyplot as plt
import os, sys, json, glob, logging

from traitlets.config.loader import Config
from lstchain.io.config import read_configuration_file
from astropy.coordinates import SkyCoord
from astropy import units as u

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# import scripts
sys.path.insert(0, os.path.join("/fefs/aswg/workspace/juan.jimenez/cosmic_ray_data_correction/scripts"))
import auxiliar as aux

# logging definition
try:
    logger
except NameError:
    logger = logging.getLogger(__name__)
    logger.addHandler(logging.StreamHandler())
    logger.setLevel(logging.INFO)

Defining all paths and variables

In [2]:
######################################
# Paths and directories
root_dl2 = "/fefs/aswg/workspace/abelardo.moralejo/Crab_performance_paper/data_v0.9.9/DL2/"
root_mc  = "/fefs/aswg/data/mc/DL2/AllSky/20221027_v0.9.9_crab_tuned/TestingDataset/dec_2276/"

path_irf = "/fefs/aswg/workspace/juan.jimenez/data/cosmic_ray_data_correction/performance_paper_data/irfs"
path_dl3 = "/fefs/aswg/workspace/juan.jimenez/data/cosmic_ray_data_correction/performance_paper_data/dl3"

config_file = "/fefs/aswg/workspace/juan.jimenez/cosmic_ray_data_correction/analysis_first_corrections/config/dl2_dl3_config.json"

# output name
fname_dict = "objects/dict_dl2_dl3_MCs.json"


######################################
# Avoid repeating long processes
compute_irf = False
compute_dl3 = False

######################################
# Source info
source_name = "Crab"

######################################
# Reading the configuration file
config = Config(read_configuration_file(config_file))
config

{'EventSelector': {'filters': {'intensity': [80, inf],
   'width': [0, inf],
   'length': [0, inf],
   'r': [0, 1],
   'wl': [0, 1],
   'leakage_intensity_width_2': [0, 1],
   'event_type': [32, 32]}},
 'DL3Cuts': {'min_event_p_en_bin': 100,
  'min_gh_cut': 0.1,
  'max_gh_cut': 0.95,
  'min_theta_cut': 0.05,
  'max_theta_cut': 0.32,
  'fill_theta_cut': 0.32,
  'allowed_tels': [1]},
 'DataBinning': {'true_energy_min': 0.002,
  'true_energy_max': 200,
  'true_energy_n_bins_per_decade': 5,
  'reco_energy_min': 0.002,
  'reco_energy_max': 200,
  'reco_energy_n_bins_per_decade': 5,
  'energy_migration_min': 0.2,
  'energy_migration_max': 5,
  'energy_migration_n_bins': 31,
  'fov_offset_min': 0.1,
  'fov_offset_max': 1.1,
  'fov_offset_n_edges': 9,
  'bkg_fov_offset_min': 0,
  'bkg_fov_offset_max': 10,
  'bkg_fov_offset_n_edges': 21,
  'source_offset_min': 0.0001,
  'source_offset_max': 1.0001,
  'source_offset_n_edges': 1000}}

Creating a dictionary with all runs and the dl2 directory, and theta and az of the MC used.

In [3]:
# Create folders if do not exist
aux.createdir(path_irf)
aux.createdir(path_dl3)

# --- Opening all files in the root path filtering that contain the words "process" in the dir --- #
items_in_root = os.listdir(root_dl2)
dirs_dl2 = [item for item in items_in_root if os.path.isdir(os.path.join(root_dl2, item)) if "process" in item]
dirs_dl2 = np.sort([os.path.join(root_dl2, d) for d in dirs_dl2])

logger.info(f"Searching for DL2 files at:\n{root_dl2}\n\nFound {len(dirs_dl2)} folders i.e. {len(dirs_dl2)} different MC used")

dict_runs = {}
# Looping over all folders
for i in range(len(dirs_dl2)):
        
        # Now we find the files inside each folder and keep only ".h5" files
        _files_ = os.listdir(dirs_dl2[i])
        _files_ = [os.path.join(dirs_dl2[i], f.decode('utf-8')) for f in _files_ if ".h5" in str(f)]
        # Extract run numbers
        run_nums = [int(f[-8:-3]) for f in _files_]    
        
        # Iterate over all files in each directory
        for j in range(len(_files_)):
            
            # Creating a sub-dict for each run
            tmp_dict = {}
            tmp_dict["dl2"] = _files_[j]
            tmp_dict["irf"] = os.path.join(path_irf, f"irf_{os.path.basename(dirs_dl2[i])[13:]}.fits.gz")
            
            dict_runs[int(run_nums[j])] = tmp_dict

Searching for DL2 files at:
/fefs/aswg/workspace/abelardo.moralejo/Crab_performance_paper/data_v0.9.9/DL2/

Found 6 folders i.e. 6 different MC used


Now we need to create the IRFs for all the MC files in the MC directory

In [4]:
%%time
# Correspondence between MC folder and MC file
dict_dirs_mc = {
    "process_with_mc_theta_10.0_az_102.199"   : "node_theta_10.0_az_102.199_/dl2_20221027_v0.9.9_crab_tuned_node_theta_10.0_az_102.199__merged.h5",
    "process_with_mc_theta_10.0_az_248.117"   : "node_theta_10.0_az_248.117_/dl2_20221027_v0.9.9_crab_tuned_node_theta_10.0_az_248.117__merged.h5",
    "process_with_mc_theta_23.63_az_100.758"  : "node_theta_23.630_az_100.758_/dl2_20221027_v0.9.9_crab_tuned_node_theta_23.630_az_100.758__merged.h5",
    "process_with_mc_theta_23.63_az_259.265"  : "node_theta_23.630_az_259.265_/dl2_20221027_v0.9.9_crab_tuned_node_theta_23.630_az_259.265__merged.h5",
    "process_with_mc_theta_32.059_az_102.217" : "node_theta_32.059_az_102.217_/dl2_20221027_v0.9.9_crab_tuned_node_theta_32.059_az_102.217__merged.h5",
    "process_with_mc_theta_32.059_az_248.099" : "node_theta_32.059_az_248.099_/dl2_20221027_v0.9.9_crab_tuned_node_theta_32.059_az_248.099__merged.h5",
}


for key in dict_dirs_mc.keys():
    
    logger.info(f"Creating IRF for {key[13:]}")
    
    mc_input   = os.path.join(root_mc, dict_dirs_mc[key])
    irf_output = os.path.join(path_irf, f"irf_{key[13:]}.fits.gz")
    logger.debug(f"\n{os.path.basename(mc_input)} --> {os.path.basename(irf_output)}")
    
    if compute_irf:
                
        # Creating the IRF
        !lstchain_create_irf_files \
        --input-gamma-dl2 $mc_input \
        --output-irf-file $irf_output \
        --point-like \
        --energy-dependent-gh \
        --energy-dependent-theta \
        --overwrite    

Creating IRF for mc_theta_10.0_az_102.199


2023-11-07 17:36:45,130 WARNING [ctapipe.IRFFITSWriter] (lstchain_create_irf_files.setup): Overwriting /fefs/aswg/workspace/juan.jimenez/data/lst1_systematics/_irf/irf_mc_theta_10.0_az_102.199.fits.gz


Creating IRF for mc_theta_10.0_az_248.117


Creating IRF for mc_theta_23.63_az_100.758


Creating IRF for mc_theta_23.63_az_259.265


Creating IRF for mc_theta_32.059_az_102.217


Creating IRF for mc_theta_32.059_az_248.099


CPU times: user 2.6 s, sys: 542 ms, total: 3.14 s
Wall time: 2min 29s


At the end we create the dl3 file from the dl2 file and the corresponding irf

In [5]:
%%time
# Add the irf directory to the dict
for run in dict_runs.keys():
    dict_runs[run]["dl3"] = os.path.join(path_dl3, f"dl3_LST-1.Run{run:05}.fits.gz")

# Getting coordinates of source
source_coords = SkyCoord.from_name(source_name)
source_ra  = f"{source_coords.ra.deg}deg"
source_dec = f"{source_coords.dec.deg}deg"

# Iterate over all dl2 files
for i, run in enumerate(dict_runs.keys()):

    # filenames of dl2 and irf
    input_dl2 = dict_runs[run]["dl2"]
    input_irf = dict_runs[run]["irf"]
    logger.debug(f"\n{os.path.basename(input_dl2)} + {os.path.basename(input_irf)} --> dl3")
    
    if compute_dl3:
            
        logger.info(f"Computing dl3 for Run{run},  {i/len(dict_runs)*100:.1f}%...")

        # Creating the dl3
        !lstchain_create_dl3_file \
        --input-dl2 $input_dl2 \
        --input-irf $input_irf \
        --output-dl3-path $path_dl3 \
        --source-name $source_name \
        --source-ra=$source_ra \
        --source-dec=$source_dec \
        --config $config_file \
        --overwrite
                  
if compute_dl3:
    
    logger.info(f"All dl3 files created 100%\n\n\nCreating index files...")
    
    # Creating the index file
    !lstchain_create_dl3_index_files \
    --input-dl3-dir $path_dl3 \
    --file-pattern 'dl3*.fits' \
    --overwrite
                  
    logger.info(f"Finished with the dl3 process")

Computing dl3 for Run3319,  0.0%...


Computing dl3 for Run3096,  0.9%...


Computing dl3 for Run2950,  1.7%...


Computing dl3 for Run2973,  2.6%...


Computing dl3 for Run3338,  3.4%...


...

All dl3 files created 100%


Creating index files...


Run 2914 does not contain HDU BACKGROUND
Run 2914 does not contain HDU PSF
Run 2929 does not contain HDU BACKGROUND
Run 2929 does not contain HDU PSF
Run 2930 does not contain HDU BACKGROUND
Run 2930 does not contain HDU PSF
Run 2931 does not contain HDU BACKGROUND
Run 2931 does not contain HDU PSF
Run 2932 does not contain HDU BACKGROUND
Run 2932 does not contain HDU PSF
Run 2933 does not contain HDU BACKGROUND
Run 2933 does not contain HDU PSF
Run 2934 does not contain HDU BACKGROUND
Run 2934 does not contain HDU PSF
Run 2949 does not contain HDU BACKGROUND
Run 2949 does not contain HDU PSF
Run 2950 does not contain HDU BACKGROUND
Run 2950 does not contain HDU PSF
Run 2967 does not contain HDU BACKGROUND
Run 2967 does not contain HDU PSF
Run 2968 does not contain HDU BACKGROUND
Run 2968 does not contain HDU PSF
Run 2969 does not contain HDU BACKGROUND
Run 2969 does not contain HDU PSF
Run 2970 does not contain HDU BACKGROUND
Run 2970 does not contain HDU PSF
Run 2971 does not contain

Run 7254 does not contain HDU BACKGROUND
Run 7254 does not contain HDU PSF
Run 7255 does not contain HDU BACKGROUND
Run 7255 does not contain HDU PSF
Run 7256 does not contain HDU BACKGROUND
Run 7256 does not contain HDU PSF
Run 7274 does not contain HDU BACKGROUND
Run 7274 does not contain HDU PSF
Run 7275 does not contain HDU BACKGROUND
Run 7275 does not contain HDU PSF
Run 7276 does not contain HDU BACKGROUND
Run 7276 does not contain HDU PSF
Run 7277 does not contain HDU BACKGROUND
Run 7277 does not contain HDU PSF


Finished with the dl3 process


CPU times: user 1min 56s, sys: 21.6 s, total: 2min 18s
Wall time: 1h 54min 36s


In [6]:
# Save the dict to a external object 
if compute_dl3:
    file = open(fname_dict, "w")
    json.dump(dict_runs, file)
    file.close()
else:
    with open(fname_dict, "r") as json_file:
        dict_runs = json.load(json_file)